# Personal Financial Modelling
- src: https://pbpython.com/amortization-model-revised.html

In [ ]:
import pandas as pd
from datetime import date
import numpy as np
from collections import OrderedDict
from dateutil.relativedelta import *
from datetime import datetime, timedelta
%matplotlib inline

In [ ]:
from absl import flags
from absl import app

In [ ]:
FLAGS = flags.FLAGS

In [ ]:
flags.DEFINE_float('mortgage_principal', 500000, 'Loan amount (principal)')
flags.DEFINE_float('interest_rate', 0.04, 'Interest rate for mortage amortization')
flags.DEFINE_integer('mortgage_years', 30, 'Mortgage length')
falgs.DEFINE_string('interest_type', 'fixed', 'Interest rate type - fixed or variable')
flags.DEFINE_string('variable_interest_fluct', 'conservative', 'Interest rate fluctuation / behaviour setting')
flags.DEFINE_float('additional_principal', 0, 'Additional principal payment above minimum repayment')
flags.DEFINE_integer('annual_payments', 12, 'Number of payments (and compounding events) per year')

In [ ]:
if __name__ == '__main__':
    app.run(main)

In [ ]:
def amortize(argv, principal,
             interest_rate,
             years,
             var_interest_rate = False,
             interest_rate_fluct = 'aggressive',
             addl_principal=0,
             annual_payments=12,
             start_date=date.today()):

    
    pmt = -round(np.pmt(interest_rate/annual_payments, years*annual_payments, principal), 2)
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal
    
    while end_balance > 0:
        if var_interest_rate and start_date.month == 6 and interest_rate < 0.06:
#             print('Modifying interest rate...')
            # Assumes that interest rate changes occur mid-year
            if interest_rate_fluct == 'chaotic':
                interest_rate = interest_rate * np.random.uniform(0.5, 1.5)
            if interest_rate_fluct == 'aggressive':
                interest_rate = interest_rate * np.random.uniform(0.8,1.2)
            if interest_rate_fluct == 'moderate':
                interest_rate = interest_rate * np.random.uniform(0.875,0.125)
            if interest_rate_fluct == 'conservative':
                interest_rate = interest_rate * np.random.uniform(0.95,1.05)
#             print(interest_rate)
        
        # Recalculate the interest based on the current balance
        interest = round(((interest_rate/annual_payments) * beg_balance), 2)

        # Determine payment based on whether or not this period will pay off the loan
        pmt = min(pmt, beg_balance + interest)
        principal = pmt - interest

        # Ensure additional payment gets adjusted if the loan is being paid off
        addl_principal = min(addl_principal, beg_balance - principal)
        end_balance = beg_balance - (principal + addl_principal)

        yield OrderedDict([('Month',start_date),
                           ('Period', p),
                           ('Begin Balance', beg_balance),
                           ('Payment', pmt),
                           ('Principal', principal),
                           ('Interest', interest),
                           ('Additional_Payment', addl_principal),
                           ('End Balance', end_balance)])

        # Increment the counter, balance and date
        p += 1
        start_date += relativedelta(months=1)
        beg_balance = end_balance

In [ ]:
def amortize_format(df):
    """
    Format amortize generator.
    
    """
    
    df.set_index('Month', inplace=True)
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
schedule_var = pd.DataFrame(amortize(700000, 0.04, 30, 
                                     var_interest_rate=True,
                                     interest_rate_fluct = 'chaotic',
                                     addl_principal=200,
                                     start_date=date(2019, 12,1)))

# Monte Carlo Simulation of Variable Loan

In [ ]:
def mc_sim(iters=10):
    sim_results = {i: pd.DataFrame(amortize(700000, 0.04, 30, 
                                     var_interest_rate=True,
                                     interest_rate_fluct='conservative',
                                     addl_principal=200,
                                     start_date=date(2019, 12,1)))['Interest'] for i in range(iters)}
    return sim_results

In [ ]:
for i in range(10):
    mc_sim()

# Global Date Table

In [ ]:
# Global date table is required as it's a reference of all financial information
currentDate = date.today()
print(f'Todays Date: {currentDate}')

birthdate = '1993-04-28'
age = (date.today() - datetime.strptime(birthdate, '%Y-%m-%d').date()) // timedelta(days=365.2425)
male = True
female = False
print(f'Individual Details -Birthdate: {birthdate} - Age: {age} - Gender: {male}')

# Life expectancy
# https://www.aihw.gov.au/reports/life-expectancy-death/deaths-in-australia/contents/life-expectancy
if female:
    # Life expectancy (at birth): 84.6 years
    lifeExpectancyFemale = 84.6
    daysLeftAlive = lifeExpectancyFemale - age
#     lastDate = 
if male:
    # Life expectancy (at birth): 80.4 years
    lifeExpectancyMale = 80.4
    daysLeftAlive = (lifeExpectancyMale - age) * 365.2425
    lastDate = currentDate +timedelta(days=daysLeftAlive)
    print(f'Last Date: {lastDate}')

### Creating global date table with general, constant, income

In [ ]:
monthly_income = 100000/12

In [ ]:
freqReq = 'MS'
dateList = pd.date_range(currentDate, lastDate, freq = freqReq, closed='right').strftime('%Y-%m-%d').tolist()

In [ ]:
dfInitData = np.ones(shape= (len(dateList)+1)) * monthly_income

In [ ]:
globalDateDict = dict(zip(dateList, dfInitData))

In [ ]:
dfGlobalDate = pd.DataFrame.from_dict(globalDateDict, orient='index')

In [ ]:
dfGlobalDate.columns = ['Monthly_Income']

In [ ]:
dfGlobalDate.index = pd.to_datetime(dfGlobalDate.index)

# Testing join functionality - variable loan

In [ ]:
df_dev = pd.merge(dfGlobalDate, schedule_var, how='outer', left_index=True, right_index=True)

### Calculating Monthly Disposable Income

In [ ]:
df_dev.fillna(0, inplace=True)

In [ ]:
df_dev['Monthly_Disposable_Income'] = df_dev['Monthly_Income'] - df_dev['Payment'] - df_dev['Additional_Payment']

In [ ]:
df_dev.head()

In [ ]:
df_dev.plot(y=['Monthly_Disposable_Income','Payment'], title='Temporal Cashflow')
# df_dev.plot(y=)

In [ ]:
df_dev.plot(y='Interest', title='Interest')